In [11]:
# !pip install pyodbc
# !pip install openpyxl
# !pip install excel2img
# !pip install flask

In [1]:
# Importing Libraries
import os
import sys
import pyodbc
import openpyxl
import warnings
import pythoncom
import shutil
import tempfile
import time as t
import datetime
import excel2img
import numpy as np
import pandas as pd
from flask import * 
import datetime as dt
from datetime import datetime, time
from datetime import datetime
from pandas import Timestamp
from fileinput import filename
from distutils.log import debug
import win32com.client as win32
from openpyxl.styles import Font
warnings.filterwarnings('ignore')
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill
from openpyxl.styles.colors import Color
from openpyxl.drawing.image import Image
from werkzeug.utils import secure_filename
from dateutil.relativedelta import relativedelta

In [2]:
# 
DB = {'servername': 'AW02PSQLC007',
    'database': 'India_GTSG'}
conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
rft_report = pd.read_sql_query('''SELECT * FROM [India_GTSG].[dbo].[RFT_P6]''', conn)
ut = pd.read_sql_query('''SELECT * FROM [India_GTSG].[dbo].[UT]''', conn)
calendar = pd.read_sql_query('''SELECT * FROM [India_GTSG].[dbo].[calendar]''', conn)
data = pd.read_excel('Book9.xlsx', sheet_name= 'Sheet1')
cursor = conn.cursor()    
# SQLCommand = ("INSERT INTO users(email) VALUES (?)")  
# Values = [u_email]   
# cursor.execute(SQLCommand,Values)     
conn.commit() 

In [3]:
# Popping the column to change the position
cll1 = rft_report.pop('Utilization_Target')
cll2 = rft_report.pop('Employee_code')
cll3 = rft_report.pop('Resource_Name')
cll4 = rft_report.pop('Project Number')

# Placing the pop up column at first
rft_report.insert(2, 'Utilization_Target', cll1)
# Placing the pop up column at first
rft_report.insert(2, 'Employee_code', cll2)
# Placing the pop up column at first
rft_report.insert(2, 'Resource_Name', cll3)
# Placing the pop up column at first
rft_report.insert(2, 'Project Number', cll4)

# To replace dash (-) & NaN with Zero (0)
except_columns = ['Project', 'Resource (Utilization Target %)/Job Code', 'Project Number', 'Resource_Name', 'Employee_code', 'Utilization_Target']
for column in rft_report.columns:
    if column not in except_columns:
        rft_report[column] = rft_report[column].replace('-', 0)
        rft_report[column] = rft_report[column].replace(np.NaN, 0)

# To change the dtypes of columns
except_columns = ['Project', 'Resource (Utilization Target %)/Job Code', 'Project Number', 'Resource_Name', 'Employee_code', 'Utilization_Target']
for column in rft_report.columns:
    if column not in except_columns:
        rft_report[column] = pd.to_numeric(rft_report[column], errors = 'coerce')

In [4]:
rft_report

,Project,Resource (Utilization Target %)/Job Code,Project Number,Resource_Name,Employee_code,Utilization_Target,Month-Year,Mar-24,Apr-24,May-24,...,Dec-24,Jan-25,Feb-25,Mar-25,Apr-25,May-25,Jun-25,Jul-25,Aug-25,Utilization_Target_percent
0,WWTP ENR Upgrade Design & Bid - 287099,"Warudkar, Piyush Avinash-715583 (0%)",287099,"Warudkar, Piyush Avinash",715583,0,0,3.33,7.14,7.14,...,22.03,0.44,0.47,0.58,0.47,0.47,0.56,0.44,0.42,NaN
1,WWTP ENR Upgrade Design & Bid - 287099,"Smith, Rita A-707633 (55%)",287099,"Smith, Rita A",707633,55,0,0.00,0.36,0.36,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
2,WWTP ENR Upgrade Design & Bid - 287099,"Gray, Padraic W-717067 (70%)",287099,"Gray, Padraic W",717067,70,0,32.22,22.86,22.86,...,30.52,0.67,0.70,0.88,0.70,0.70,0.84,0.67,0.63,NaN
3,WWTP ENR Upgrade Design & Bid - 287099,"Basili, Kevin M-103377 (95%)",287099,"Basili, Kevin M",103377,95,0,38.55,24.28,24.28,...,31.29,0.44,0.47,0.58,0.47,0.47,0.56,0.44,0.42,NaN
4,WWTP ENR Upgrade Design & Bid - 287099,"Crowley, Joseph D-85859 (95%)",287099,"Crowley, Joseph D",85859,95,0,58.61,61.43,61.43,...,60.91,0.89,0.94,1.17,0.94,0.94,1.12,0.89,0.84,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48545,2023 CECIL LUC INSPECTION - 292491,"Fincham, Kara E-711314 (80%)",292491,"Fincham, Kara E",711314,80,0,5.47,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
48546,6482.006 Outfall Z - 253808,"Doran, David M-705824 (97%)",253808,"Doran, David M",705824,97,0,1.10,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
48547,6482.006 Outfall Z - 253808,"Boggs, Justin S-85512 (93%)",253808,"Boggs, Justin S",85512,93,0,0.37,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
48548,6482.006 Outfall Z - 253808,"Mueller, Edmund C-702661 (97%)",253808,"Mueller, Edmund C",702661,97,0,0.74,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN


In [5]:
rft_report[rft_report['Resource (Utilization Target %)/Job Code'] == 'Stalnaker, Todd J-85420 (30%)']

,Project,Resource (Utilization Target %)/Job Code,Project Number,Resource_Name,Employee_code,Utilization_Target,Month-Year,Mar-24,Apr-24,May-24,...,Dec-24,Jan-25,Feb-25,Mar-25,Apr-25,May-25,Jun-25,Jul-25,Aug-25,Utilization_Target_percent
380,OZONE SYSTEM REHAB - 276119,"Stalnaker, Todd J-85420 (30%)",276119,"Stalnaker, Todd J",85420,30,0,782.05,464.31,440.64,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4106,SOUTH LAKE WTP - 248929,"Stalnaker, Todd J-85420 (30%)",248929,"Stalnaker, Todd J",85420,30,0,28.01,22.41,20.53,...,0.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5020,TANNEHILL STORMWATER & WW PH I - 284015,"Stalnaker, Todd J-85420 (30%)",284015,"Stalnaker, Todd J",85420,30,0,0.00,0.00,0.00,...,0.00,0.0,0.0,10.0,0.0,0.0,40.0,0.0,0.0,NaN
5981,BULL HIDE CREEK WWTP EXP - 256007,"Stalnaker, Todd J-85420 (30%)",256007,"Stalnaker, Todd J",85420,30,0,2.13,1.70,0.17,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
6298,Floyd Branch RWWTP Peak - 274547,"Stalnaker, Todd J-85420 (30%)",274547,"Stalnaker, Todd J",85420,30,0,13.48,10.79,10.79,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39415,Bonham & Tawakoni Caustic Stdy - 290815,"Stalnaker, Todd J-85420 (30%)",290815,"Stalnaker, Todd J",85420,30,0,6.66,2.40,0.00,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
40378,2346.004 Edinburg PS Leeve CM - 282860,"Stalnaker, Todd J-85420 (30%)",282860,"Stalnaker, Todd J",85420,30,0,0.29,5.78,5.78,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
41797,HEATH DE LEON 30 INCH WTM - 272978,"Stalnaker, Todd J-85420 (30%)",272978,"Stalnaker, Todd J",85420,30,0,6.00,0.00,0.00,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
44041,TIERRA ESTE ZARAGOSA WW MAIN - 247791,"Stalnaker, Todd J-85420 (30%)",247791,"Stalnaker, Todd J",85420,30,0,11.61,0.39,0.00,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [6]:
# Remove extra space
rft_report['Employee_code'] = rft_report['Employee_code'].apply(str.strip)

# Dropping unwanted columns
data1 = data.drop(['Activity Name', 'Curve', 'Spreadsheet Field', 'Remaining Units'], axis = 1)

#changing the format of date and adding it in a new column
calendar['Month_Year'] = calendar['GL Period'].apply(lambda x: x.strftime('%b-%y')) 
# Extracting Month columns 
data2 = data1.iloc[:, 1:]

x = data2.columns[0]
x2 = x.strftime('%Y-%m-%d %H:%M:%S')
x3 = x2.replace(' 00:00:00', '')
# Rename a column using the .rename() method
data2.rename(columns={x2: x3}, inplace=True)

In [7]:
# Fix time frame according to RFT (18 months)

# To get currentatime
current_date = dt.datetime.now()
# current_date = '2023-10-29'
# Select dates less than current time
current_date_less = calendar[calendar['W/E Date'] <= current_date]
# Convert the datatype
current_date_less['W/E Date'] = current_date_less['W/E Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
# Select last date which is just less than current date - start date
start_d = current_date_less.tail(1)

# Store start date in a list
start = start_d['GL Period'].tolist()
start_dt = start[0]
varr = start_dt.strftime('%Y-%m-%d')
start_date = varr.replace(' 00:00:00', '')

# Convert the string to a datetime object
start_date = dt.datetime.strptime(start_date, '%Y-%m-%d').date()

# Add 18 months using relativedelta
end_date = start_date + relativedelta(months=17)

# Convert the future date back to a string if needed
end_date = end_date.strftime('%Y-%m-%d')

var1 = data2.columns[0]
var2 = var1.strftime('%Y-%m-%d')

# Specify the format of the date string
date_format = '%Y-%m-%d'  # This format corresponds to 'YYYY-MM-DD'

# Convert the string to a datetime.date object
date_obj = datetime.strptime(var2, date_format).date()
end_date = datetime.strptime(end_date, date_format).date()

# Create a time object with the desired time (e.g., midnight)
time_obj = time(0, 0)  # This represents 00:00:00 (midnight)

# Combine the date and time to create a datetime.datetime object
datetime_obj = datetime.combine(date_obj, time_obj)
datetime_start = datetime.combine(start_date, time_obj)
datetime_end = datetime.combine(end_date, time_obj)

In [8]:
if datetime_start != datetime_obj:
    if datetime_start < datetime_obj:
        # Selecting columns mentioned in P6
        calendar = calendar[calendar['W/E Date'].isin(data2.columns)] 
        calender = calendar[calendar['W/E Date'] >= datetime_obj]
        # Reindexing
        calender = calender.reset_index(drop = True)
        # Select rows within the date range
        final_calender = calender[(calender['W/E Date'] >= datetime_obj) & (calender['W/E Date'] <= datetime_end)]
        column_list = final_calender['W/E Date'].tolist()
        # Convert the list of timestamps to a list of strings
        col_list = [timestamp.strftime('%Y-%m-%d') for timestamp in column_list]
        data2 = data2[col_list]
        #extracting all month-year values
        month = final_calender['Month_Year'].values
        data5 = data2.set_axis(month, axis = 'columns')
    else: 
        # Selecting columns mentioned in P6
        calendar = calendar[calendar['W/E Date'].isin(data2.columns)] 
        calender = calendar[calendar['GL Period'] >= datetime_start]
        # Reindexing
        calender = calender.reset_index(drop = True)
        # Select rows within the date range
        final_calender = calender[(calender['GL Period'] >= datetime_start) & (calender['GL Period'] <= datetime_end)]
        column_list = final_calender['W/E Date'].tolist()
        data2 = data2[column_list]
        #extracting all month-year values
        month = final_calender['Month_Year'].values
        data5 = data2.set_axis(month, axis = 'columns')
else: 
    # Selecting columns mentioned in P6
    calendar = calendar[calendar['W/E Date'].isin(data2.columns)] 
    calender = calendar[calendar['W/E Date'] >= datetime_obj]
    # Reindexing
    calender = calender.reset_index(drop = True)
    # Select rows within the date range
    final_calender = calender[(calender['W/E Date'] >= datetime_obj) & (calender['W/E Date'] <= datetime_end)]
    column_list = final_calènder['W/E Date'].tolist()
    data2 = data2[column_list]
    #extracting all month-year values
    month = final_calender['Month_Year'].values
    data5 = data2.set_axis(month, axis = 'columns')

In [9]:
data5

,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,175.0,136.0,109.0,93.0,53.0,1.0,NaN,NaN,NaN
1,13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN
2,13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN
3,13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN
4,13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
229,13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN
230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Joining df with p_six
data5.insert(loc = 0, column = 'Activity ID', value = data['Activity ID'])

# Extracting rows with 'Employee Number'
code = data5[data5['Activity ID'].str.contains("Employee")]
# Creating new column named 'Resource Name' by replacing Resource name from col 'Activity ID'
code['Employee Number'] = code['Activity ID'].str.replace('Employee Number: ', '')

# Dropping unwanted columns
dataset = code.drop('Activity ID', axis = 1)

# Popping the column to change the position
val1 = dataset.pop('Employee Number')

# Placing the pop up column at first
dataset.insert(0, 'Employee Number', val1)

# Reseting Index
data6 = dataset.reset_index()

# Dropping unwanted columns
data7 = data6.drop(['index'], axis = 1)

# Changing column name '0' ---> 'Project Number'
data7.rename(columns = {'Employee Number': 'Employee_code'}, inplace = True)

# Remove extra space
data7['Employee_code'] = data7['Employee_code'].apply(str.strip)

# Extracting rows with 'WBS'
wbs = data5[data5['Activity ID'].str.contains("WBS: ")]

# Extracting numbers from main column
wbs['num'] = wbs['Activity ID'].str.findall(r'[0-9]+')   #output: [285241, 01, 01], want only oth index i.e., project number

# Removing brackets from each row
wbs["Project Number"] = wbs["num"].str.get(0)

# Project Number in p6
global pn_list
pn_list = list(wbs['Project Number'].unique())
# List of unique Resource Name in p6 data
unique_resource = list(data7['Employee_code'].unique())
unique_resource = [i.strip() for i in unique_resource]

if 'No Employee Number' in unique_resource:
    unique_resource.remove('No Employee Number')

In [11]:
data7

,Employee_code,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,710570,13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN
1,709880,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,714679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50480,14.0,11.0,11.0,13.0,4.0,NaN,NaN,NaN,NaN
4,706796,26.0,22.0,22.0,31.0,21.0,NaN,NaN,NaN,NaN
5,114058,15.0,12.0,10.0,6.0,4.0,0.0,NaN,NaN,NaN
6,24533,2.0,2.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN
7,23496,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
8,706873,3.0,2.0,2.0,2.0,1.0,0.0,NaN,NaN,NaN
9,87679,6.0,4.0,4.0,4.0,3.0,0.0,NaN,NaN,NaN


In [12]:
# Extracting rows with 'Resource Name'
pattern = 'Resource|Employee'

code = data5[data5['Activity ID'].str.contains(pattern)]

# Creating new column named 'Resource Name' by replacing Resource name from col 'Activity ID'
code['Info'] = code['Activity ID'].str.replace('Employee Number:', '')
code['Info'] = code['Info'].str.replace('Resource Name: ', '')

# Dropping unwanted columns
codeset = code.drop('Activity ID', axis = 1)
codeset = codeset.reset_index()

codeset = codeset.drop(['index'], axis = 1)

ll2 = codeset.pop('Info')

# Placing the pop up column at first
codeset.insert(0, 'Info', ll2)

# Apply strip to the 'names' column
codeset['Info'] = codeset['Info'].str.strip()

In [13]:
codeset

,Info,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,"Camp, William D",13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN
1,710570,13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN
2,"Punuru, Nikhila",0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,709880,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Amador-French, Christopher G",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,714679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Brick, William D",14.0,11.0,11.0,13.0,4.0,NaN,NaN,NaN,NaN
7,50480,14.0,11.0,11.0,13.0,4.0,NaN,NaN,NaN,NaN
8,"Cailer, Noel",26.0,22.0,22.0,31.0,21.0,NaN,NaN,NaN,NaN
9,706796,26.0,22.0,22.0,31.0,21.0,NaN,NaN,NaN,NaN


In [14]:
# Extracting rows with 'Resource Name'
pattern = 'Resource'

cod = data5[data5['Activity ID'].str.contains(pattern)]

# Creating new column named 'Resource Name' by replacing Resource name from col 'Activity ID'
cod['Info'] = cod['Activity ID'].str.replace('Resource Name: ', '')

# Dropping unwanted columns
codes = cod.drop('Activity ID', axis = 1)
codes = codes.reset_index()

codes = codes.drop(['index'], axis = 1)

lo2 = codes.pop('Info')

# Placing the pop up column at first
codes.insert(0, 'Info', lo2)

# Apply strip to the 'names' column
codes['Info'] = codes['Info'].str.strip()

# Dictionary to map old column names to new column names
new_columns = {'Info': 'Resource_Name'}

# Rename columns using the 'rename' method
codes = codes.rename(columns=new_columns)

In [15]:
codes


,Resource_Name,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,"Camp, William D",13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN
1,"Punuru, Nikhila",0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Amador-French, Christopher G",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Brick, William D",14.0,11.0,11.0,13.0,4.0,NaN,NaN,NaN,NaN
4,"Cailer, Noel",26.0,22.0,22.0,31.0,21.0,NaN,NaN,NaN,NaN
5,"Cogger, Brittany A",15.0,12.0,10.0,6.0,4.0,0.0,NaN,NaN,NaN
6,"Davis, Thomas W",2.0,2.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN
7,"Deboer, Shana L",3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
8,"Dunbar, Caroline J",3.0,2.0,2.0,2.0,1.0,0.0,NaN,NaN,NaN
9,"Ebersold, David B",6.0,4.0,4.0,4.0,3.0,0.0,NaN,NaN,NaN


In [16]:
# Select p6 resource from rft for 'Employee code and UT'
rft1 = rft_report[rft_report['Employee_code'].isin(unique_resource)]

# Get unique list of code and ut for resource in p6
rft2 = rft1[['Resource_Name', 'Employee_code', 'Utilization_Target']].drop_duplicates()
rft2 = rft2.reset_index()

# Dropping unwanted columns
rft3 = rft2.drop(['index'], axis = 1)

# Merging data7 & rft3 for 'Employee_code' & 'Utilization_Target'
merged_df = pd.merge(data7, rft3, on = 'Employee_code', how = 'right')

In [17]:
merged_df

,Employee_code,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24,Resource_Name,Utilization_Target
0,706796,26.0,22.0,22.0,31.0,21.0,NaN,NaN,NaN,NaN,"Cailer, Noel",70
1,706527,13.0,10.0,6.0,3.0,1.0,0.0,NaN,NaN,NaN,"Jembere, Hana A",96
2,84513,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,"Frizzell, Carl S",77
3,40037,9.0,6.0,5.0,4.0,3.0,NaN,NaN,NaN,NaN,"Taurasi, Paul M",85
4,718311,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,"Patel, Radhika N",94
5,25974,4.0,3.0,3.0,2.0,1.0,0.0,NaN,NaN,NaN,"Wood, Christine M",60
6,96438,5.0,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"Pagaduan, Charles",90
7,84378,14.0,11.0,8.0,5.0,4.0,0.0,NaN,NaN,NaN,"Ott, Christopher A",82
8,87248,16.0,12.0,7.0,1.0,NaN,NaN,NaN,NaN,NaN,"McDaniel, Charles K",94
9,713496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Lanza, Heather A",75


In [18]:
# Popping the column to change the position
vll1 = merged_df.pop('Utilization_Target')
vll2 = merged_df.pop('Resource_Name')

# Placing the pop up column at first
merged_df.insert(1, 'Utilization_Target', vll1)
# Placing the pop up column at first
merged_df.insert(0, 'Resource_Name', vll2)

# To replace dash (-) & NaN with Zero (0)
except_col = ['Resource_Name', 'Employee_code', 'Utilization_Target Name']
for column in merged_df.columns:
    if column not in except_col:
        #data9[column] = data9[column].replace('-', 0)
        merged_df[column] = merged_df[column].replace(np.NaN, 0)

# Convert data types for columns except the excluded ones
exclude_columns = ['Resource_Name', 'Employee_code', 'Utilization_Target Name']
for column in merged_df.columns:
    if column not in exclude_columns:
        merged_df[column] = pd.to_numeric(merged_df[column], errors='coerce')  # You can use other conversion functions if needed

In [19]:
merged_df

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,"Cailer, Noel",706796,70,26.0,22.0,22.0,31.0,21.0,0.0,0.0,0.0,0.0
1,"Jembere, Hana A",706527,96,13.0,10.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0
2,"Frizzell, Carl S",84513,77,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,"Taurasi, Paul M",40037,85,9.0,6.0,5.0,4.0,3.0,0.0,0.0,0.0,0.0
4,"Patel, Radhika N",718311,94,3.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"Wood, Christine M",25974,60,4.0,3.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0
6,"Pagaduan, Charles",96438,90,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"Ott, Christopher A",84378,82,14.0,11.0,8.0,5.0,4.0,0.0,0.0,0.0,0.0
8,"McDaniel, Charles K",87248,94,16.0,12.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0
9,"Lanza, Heather A",713496,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
 # Popping the column to change the position
cal1 = ut.pop('Utilization_Target')
cal2 = ut.pop('Employee_code')
cal3 = ut.pop('Resource_Name')

# Placing the pop up column at first
ut.insert(1, 'Utilization_Target', cal1)
# Placing the pop up column at first
ut.insert(1, 'Employee_code', cal2)
# Placing the pop up column at first
ut.insert(1, 'Resource_Name', cal3)
ut = ut.drop('Utilization_Target_percent', axis = 1)

## Select specific uts
sel = ut[ut['Employee_code'].isin(unique_resource)]

# Ut with only combine columns
ut_com = sel[merged_df.columns]

# RESET INDEX
ut_com = ut_com.reset_index(drop = True)

In [21]:
# set2

In [22]:
rft_report[rft_report['Resource_Name'] == 'Cailer, Noel']['Mar-24'].sum()

480.64

In [23]:
grouped_rft[grouped_rft['Resource_Name'] == 'Cailer, Noel']['Mar-24'].sum()

NameError: name 'grouped_rft' is not defined

In [ ]:
set2[set2['Resource_Name'] == 'Cailer, Noel']['Mar-24'].sum()

In [24]:
pn_list

['265573', '284072']

In [25]:
# Groupby RFT_REPORT based on 'Resource Name' & 'Project Number'
grouped = rft_report.groupby(['Resource_Name', 'Employee_code', 'Utilization_Target','Project Number']).sum()

# Redexing to apply filter on 'Resource Name'
grouped_rft = grouped.reset_index()

# P6 in RFT
set1 = grouped_rft[grouped_rft['Employee_code'].isin(unique_resource)]

# Selecting rft based on Project number not in final_p6 (NOT EXIST/EXIST CASE)
set2 = set1[~set1['Project Number'].isin(pn_list)]

# set2 with only merged_df columns ---index issue
set2 = set2[merged_df.columns]

# Groupby RFT_REPORT based on 'Resource Name' & 'Project Number'
set3 = set2.groupby(['Resource_Name', 'Employee_code', 'Utilization_Target']).sum()
set3 = set3.reset_index()

In [26]:
set2[set2['Resource_Name'] == 'Cailer, Noel']['Mar-24'].sum()

447.58

In [27]:
## 1. Conbine data9 and set3
combine = merged_df.set_index(['Resource_Name','Employee_code','Utilization_Target']).add(set3.set_index(['Resource_Name','Employee_code','Utilization_Target']), fill_value = 0.0).reset_index()

# List of unique Resource Name in p6 data
listrft = list(combine['Employee_code'].unique())
listrft = [i.strip() for i in listrft]
combine1 = ut_com.set_index(['Resource_Name','Employee_code','Utilization_Target']).subtract(combine.set_index(['Resource_Name','Employee_code','Utilization_Target']), fill_value=0).reset_index()

# List of missing resource name (not in rft)
resource = list(combine1['Employee_code'].unique())
resource = [i.strip() for i in resource]

# Not in rft
missing_resource = set(unique_resource) - set(resource)
miss_df = data7[data7['Employee_code'].isin(missing_resource)]

In [28]:
# miss_df
# data7[data7['Employee_code'] == '713625'] 

In [29]:
# combine1[combine1['Employee_code'] == '713625'] 
combine1

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,"Amador-French, Christopher G",714679,93,54.90,34.81,27.38,38.19,37.52,33.99,135.11,136.03,138.20
1,"Brick, William D",50480,87,-209.70,-118.90,-92.42,-84.54,81.53,73.71,102.18,88.69,87.45
2,"Cailer, Noel",706796,70,-333.58,-95.57,-13.65,-79.04,3.02,32.25,-94.64,-113.27,-4.86
3,"Cogger, Brittany A",114058,92,25.17,2.46,24.51,80.83,8.77,12.53,20.35,71.87,88.38
4,"Davis, Thomas W",24533,90,24.35,16.21,17.66,28.80,26.16,20.03,43.67,33.12,33.12
5,"Deboer, Shana L",23496,35,33.83,19.29,13.03,17.44,6.64,1.65,33.39,34.85,33.36
6,"Dunbar, Caroline J",706873,63,-321.49,-185.54,-155.07,-191.91,-100.43,-94.69,-102.10,-68.25,-61.75
7,"Ebersold, David B",87679,15,-22.19,-20.91,-7.97,-2.18,14.78,15.77,19.85,17.51,9.58
8,"Frizzell, Carl S",84513,77,-394.59,-342.07,-342.53,-221.74,-138.33,-169.33,-168.25,-165.40,-119.07
9,"Higuchi, Michael J",709862,55,65.20,53.56,56.40,77.54,70.14,73.45,93.26,74.25,74.90


In [30]:
match = {'Resource_Name': [],
         'Employee_code': [] }

for idx, i in enumerate(missing_resource):
    # Use isin() to find the matching row index
    matching_index = codeset[codeset['Info'].isin([i])].index

    try:
        # Extract the row immediately following the matching index
        if not matching_index.empty:
            following_row_index = matching_index[0] - 1      # code row
            following_row = codeset.iloc[following_row_index]   # code numer

            match['Employee_code'].append(codeset.iloc[matching_index[0]]['Info'])
            match['Resource_Name'].append(following_row['Info'])
        else:
            continue
    except IndexError:
        match['Employee_code'].append(i)
        match['Resource_Name'].append('-')
        
df = pd.DataFrame(match)
def concat(df1, df2):
    try:
        if df1.empty and df2.empty:
            return df2
        else:
            # Merge DataFrames based on the 'ID' column
            missdf = pd.merge(df1, df2, on='Employee_code', how='inner')
            return missdf
    except:
        return df2
missdf = concat(df, miss_df)

In [31]:
missdf

,Resource_Name,Employee_code,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,"Punuru, Nikhila",709880,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Pillsbury, Lana M",717389,6.0,5.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN
2,"Camp, William D",710570,13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN


In [32]:
def con(df1, df2):
    try:
        if df2.empty:
            return df1
        else:
            # Merging data7 & rft3 for 'Employee_code' & 'Utilization_Target'
            combb = pd.concat([df1, df2], axis = 0)
            return combb
    except AttributeError:
        if df2.empty:
            return df1

combb = con(combine1, missdf)

# To replace dash (-) & NaN with Zero (0)
incl_col = ['Resource_Name', 'Employee_code', 'Utilization_Target Name']
for column in combb.columns:
    if column in incl_col:
        #data9[column] = data9[column].replace('-', 0)
        combb[column] = combb[column].replace(np.NaN, '')
        
combb['Utilization_Target'] = combb['Utilization_Target'].astype(str)
combb['Employee_code'] = combb['Employee_code'].astype(str)

In [33]:
result = {'Resource_Name': [],
         'Employee_code': [] }

# Use isin() to find the matching row index
matching_index = codeset[codeset['Info']== 'No Employee Number'].index
for idx, i in enumerate(matching_index):
    try:
        # Extract the row immediately following the matching index
        if not matching_index.empty:
            following_row_index = i - 1      # code row
            following_row = codeset.iloc[following_row_index]   # code numer

            result['Employee_code'].append('-')
            result['Resource_Name'].append(following_row['Info'])
        else:
            continue
    except IndexError:
        result['Employee_code'].append(i)
        result['Resource_Name'].append('-')
        
dff = pd.DataFrame(result)

def contt(df1, df2):
    try:
        if df2.empty:
            return df2
        else:
            # Merging data7 & rft3 for 'Employee_code' & 'Utilization_Target'
            mrg = pd.merge(df1, df2, on='Resource_Name', how='inner')
            return mrg
    except AttributeError:
        if df2.empty:
            return df2

mrg1 = contt(codes, dff)

In [34]:
def cont(df1, df2):
    try:
        if df1.empty:
            return df2
        elif df1.empty and df2.empty:
            return df1
        elif df2.empty:
            return df1
        else:
            # Merging data7 & rft3 for 'Employee_code' & 'Utilization_Target'
            mrg = pd.concat([df1, df2], axis = 0)
            return mrg
    except AttributeError:
        if df1.empty:
            return df2

mrg = cont(missdf, mrg1)

# # Merging data7 & rft3 for 'Employee_code' & 'Utilization_Target'
# mrg = pd.concat([missdf, result], axis = 0)
# def asterik(df):
#     try:
#         if df.empty :
#             return df
#         else:
#             # # Add '*' after each value in columns 'A' and 'B'
#             df['Resource_Name'] = df['Resource_Name'].astype(str) + '*'
#             return mrg
#     except AttributeError:
#         if df.empty:
#             return df

# mrg = asterik(mrg)

In [35]:
def vrt(df1, df2):
    try:
        if df2.empty:
            return df1
        else:
            # Merging data7 & rft3 for 'Employee_code' & 'Utilization_Target'
            comb = pd.concat([df1, df2], axis = 0)
            comb['Utilization_Target'] = comb['Utilization_Target'].astype(str)
            comb['Employee_code'] = comb['Employee_code'].astype(str)
            return comb
    except AttributeError:
        if df2.empty:
            return df1

comb = vrt(combine1, mrg)

In [36]:
# To replace dash (-) & NaN with Zero (0)
except_col = ['Employee_code', 'Utilization_Target']
for column in comb.columns:
    if column in except_col:
        #data9[column] = data9[column].replace('-', 0)
        comb[column] = comb[column].replace('nan', '-')
comb = comb.round(0)
comb = comb.reset_index()
comb = comb.drop('index', axis=1)

def highlight_max(cell_value):
    highlight = 'background-color: tomato;'
    color = 'background-color: lightblue;'
    default = ''
    if type(cell_value) in [float]:
        if cell_value < 0:
            return highlight
        else:
            return color
    else:
        if '*' in cell_value:
            return 'color:red'
    return default

# # Apply the custom styling to the entire DataFrame
# # Apply color to combine
final_data = comb.style.applymap(highlight_max).format(precision = 0)
test = comb.copy()

In [37]:
# AVAILABLE RESOURCES
test = test[(test[test.columns[3:]] >= 0).all(axis=1)]
pos_resources = test['Employee_code'].unique()

# Selecting pos_resources from arr
neg_info = combine1[~combine1['Employee_code'].isin(pos_resources)]
neg_info = neg_info.reset_index(drop=True)

In [38]:
# Calling from rft: set3
rft_set4 = set3[set3['Employee_code'].isin(combine1['Employee_code'].unique())].reset_index(drop=True)

# Calling from p6: data9
data10 = merged_df[merged_df['Employee_code'].isin(combine1['Employee_code'].unique())].reset_index(drop=True)
data10 = data10.sort_values(by = 'Employee_code')

In [39]:
rft_set4

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,"Amador-French, Christopher G",714679,93,131.10,113.99,121.42,147.81,111.28,114.81,50.89,12.77,10.60
1,"Brick, William D",50480,87,369.70,247.10,220.62,245.54,53.67,65.49,71.82,50.51,51.75
2,"Cailer, Noel",706796,70,447.58,185.57,103.65,188.04,87.98,79.75,234.64,225.27,116.86
3,"Cogger, Brittany A",114058,92,143.83,132.74,112.69,97.17,134.43,134.67,163.65,75.33,58.82
4,"Davis, Thomas W",24533,90,153.65,125.79,124.34,149.20,117.84,123.97,136.33,110.88,110.88
5,"Deboer, Shana L",23496,35,33.17,34.71,40.97,52.56,49.36,54.35,36.61,21.15,22.64
6,"Dunbar, Caroline J",706873,63,444.49,284.34,253.87,315.91,200.23,195.49,228.10,169.05,162.55
7,"Ebersold, David B",87679,15,46.19,40.91,27.97,28.18,6.22,8.23,10.15,6.49,14.42
8,"Frizzell, Carl S",84513,77,547.59,464.27,464.73,374.74,260.53,292.53,322.25,288.60,242.27
9,"Higuchi, Michael J",709862,55,42.80,32.44,29.60,31.46,17.86,14.55,16.74,13.75,13.10


In [40]:
# Calling from ut: ut_com
ut1 = ut_com[ut_com['Employee_code'].isin(combine1['Employee_code'].unique())].reset_index(drop=True)

## 1. Combine data10 and rft_set4
av_hours = ut1.set_index(['Resource_Name','Employee_code','Utilization_Target']).subtract(rft_set4.set_index(['Resource_Name','Employee_code','Utilization_Target']), fill_value=0).reset_index()
av_hours = av_hours.sort_values(by = 'Employee_code')

In [41]:
av_hours

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
3,"Cogger, Brittany A",114058,92,40.17,14.46,34.51,86.83,12.77,12.53,20.35,71.87,88.38
5,"Deboer, Shana L",23496,35,36.83,21.29,15.03,17.44,6.64,1.65,33.39,34.85,33.36
4,"Davis, Thomas W",24533,90,26.35,18.21,19.66,30.80,26.16,20.03,43.67,33.12,33.12
23,"Wood, Christine M",25974,60,-107.03,-61.51,-63.94,-44.66,-8.09,5.75,19.48,17.86,27.27
21,"Taurasi, Paul M",40037,85,-270.68,-238.64,-169.52,-146.11,-112.08,-87.62,-63.45,-4.96,34.05
1,"Brick, William D",50480,87,-195.70,-107.90,-81.42,-71.54,85.53,73.71,102.18,88.69,87.45
10,"Jembere, Hana A",706527,96,-42.59,-99.77,57.26,96.73,75.18,81.63,112.17,90.81,90.72
2,"Cailer, Noel",706796,70,-307.58,-73.57,8.35,-48.04,24.02,32.25,-94.64,-113.27,-4.86
6,"Dunbar, Caroline J",706873,63,-318.49,-183.54,-153.07,-189.91,-99.43,-94.69,-102.10,-68.25,-61.75
11,"Kwong, Alison N",708617,10,6.14,4.03,3.03,10.28,8.83,8.47,10.95,9.09,9.12


In [42]:
mssdf = missdf.copy()

In [43]:
# add percent symbol in utilization target
# mssdf['Utilization_Target'] = 111

In [44]:
mssdf['Utilization_Target'] = 0

In [45]:
# Popping the column to change the position
mdf = mssdf.pop('Utilization_Target')

# Placing the pop up column at first
mssdf.insert(2, 'Utilization_Target', mdf)

In [46]:
mssdf

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,"Punuru, Nikhila",709880,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Pillsbury, Lana M",717389,0,6.0,5.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN
2,"Camp, William D",710570,0,13.0,11.0,11.0,15.0,10.0,NaN,NaN,NaN,NaN


In [47]:
data11 = pd.concat([data10,mssdf], axis=0, ignore_index=True)
data11

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,"Cogger, Brittany A",114058,92,15.0,12.0,10.0,6.0,4.0,0.0,0.0,0.0,0.0
1,"Deboer, Shana L",23496,35,3.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Davis, Thomas W",24533,90,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
3,"Wood, Christine M",25974,60,4.0,3.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0
4,"Taurasi, Paul M",40037,85,9.0,6.0,5.0,4.0,3.0,0.0,0.0,0.0,0.0
5,"Brick, William D",50480,87,14.0,11.0,11.0,13.0,4.0,0.0,0.0,0.0,0.0
6,"Jembere, Hana A",706527,96,13.0,10.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0
7,"Cailer, Noel",706796,70,26.0,22.0,22.0,31.0,21.0,0.0,0.0,0.0,0.0
8,"Dunbar, Caroline J",706873,63,3.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0
9,"Kwong, Alison N",708617,10,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# Change all values to 0 in all columns except 'ut' and 'employee'
mssdf.loc[:, ~mssdf.columns.isin(['Resource_Name','Employee_code','Utilization_Target'])] = 0

In [49]:
av_hours1 = pd.concat([mssdf, av_hours], axis=0, ignore_index=True)

In [50]:
av_hours1=av_hours1.sort_values('Employee_code')
av_hours1

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
3,"Cogger, Brittany A",114058,92,40.17,14.46,34.51,86.83,12.77,12.53,20.35,71.87,88.38
4,"Deboer, Shana L",23496,35,36.83,21.29,15.03,17.44,6.64,1.65,33.39,34.85,33.36
5,"Davis, Thomas W",24533,90,26.35,18.21,19.66,30.80,26.16,20.03,43.67,33.12,33.12
6,"Wood, Christine M",25974,60,-107.03,-61.51,-63.94,-44.66,-8.09,5.75,19.48,17.86,27.27
7,"Taurasi, Paul M",40037,85,-270.68,-238.64,-169.52,-146.11,-112.08,-87.62,-63.45,-4.96,34.05
8,"Brick, William D",50480,87,-195.70,-107.90,-81.42,-71.54,85.53,73.71,102.18,88.69,87.45
9,"Jembere, Hana A",706527,96,-42.59,-99.77,57.26,96.73,75.18,81.63,112.17,90.81,90.72
10,"Cailer, Noel",706796,70,-307.58,-73.57,8.35,-48.04,24.02,32.25,-94.64,-113.27,-4.86
11,"Dunbar, Caroline J",706873,63,-318.49,-183.54,-153.07,-189.91,-99.43,-94.69,-102.10,-68.25,-61.75
12,"Kwong, Alison N",708617,10,6.14,4.03,3.03,10.28,8.83,8.47,10.95,9.09,9.12


In [51]:
data11=data11.sort_values('Employee_code')
data11

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
0,"Cogger, Brittany A",114058,92,15.0,12.0,10.0,6.0,4.0,0.0,0.0,0.0,0.0
1,"Deboer, Shana L",23496,35,3.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Davis, Thomas W",24533,90,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
3,"Wood, Christine M",25974,60,4.0,3.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0
4,"Taurasi, Paul M",40037,85,9.0,6.0,5.0,4.0,3.0,0.0,0.0,0.0,0.0
5,"Brick, William D",50480,87,14.0,11.0,11.0,13.0,4.0,0.0,0.0,0.0,0.0
6,"Jembere, Hana A",706527,96,13.0,10.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0
7,"Cailer, Noel",706796,70,26.0,22.0,22.0,31.0,21.0,0.0,0.0,0.0,0.0
8,"Dunbar, Caroline J",706873,63,3.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0
9,"Kwong, Alison N",708617,10,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# Compare av_hours & data10
compa = data11.set_index(['Resource_Name','Employee_code','Utilization_Target']).compare(av_hours1.set_index(['Resource_Name','Employee_code','Utilization_Target']), align_axis = 1).rename(columns={'self': 'P6 Hrs', 'other': 'Avail Hrs'}, level=1).reset_index()

# Replace Nan with 0
comp = compa.replace(np.NaN, 0)
comp=comp.round(0)

In [53]:
comp

Resource_Name Employee_code Utilization_Target Mar-24  \
                                                                  P6 Hrs   
0             Cogger, Brittany A        114058                 92   15.0   
1                Deboer, Shana L         23496                 35    3.0   
2                Davis, Thomas W         24533                 90    2.0   
3              Wood, Christine M         25974                 60    4.0   
4                Taurasi, Paul M         40037                 85    9.0   
5               Brick, William D         50480                 87   14.0   
6                Jembere, Hana A        706527                 96   13.0   
7                   Cailer, Noel        706796                 70   26.0   
8             Dunbar, Caroline J        706873                 63    3.0   
9                Kwong, Alison N        708617                 10    3.0   
10            Higuchi, Michael J        709862                 55    2.0   
11               Punuru, Nikhila        709880                  0    0.0   
12               Camp, William D        710570                  0   13.0   
13              Lanza, Heather A        713496                 75    0.0   
14              Olmsted, Jenny L        713559                 91    0.0   
15  Amador-French, Christopher G        714679                 93    0.0   
16             Supler, Kaitlyn R        715332                 97   10.0   
17             Pillsbury, Lana M        717389                  0    6.0   
18              Patel, Radhika N        718311                 94    3.0   
19       Perez, Severo M (Moses)         83685                 95    6.0   
20            Ott, Christopher A         84378                 82   14.0   
21              Frizzell, Carl S         84513                 77    1.0   
22            Wilkins, Suzanne M         84518                 91    1.0   
23           McDaniel, Charles K         87248                 94   16.0   
24             Ebersold, David B         87679                 15    6.0   
25         Macbeth, Dr. Tamzen W         88612                 35    0.0   
26             Pagaduan, Charles         96438                 90    5.0   

             Apr-24           May-24           Jun-24  ... Jul-24            \
   Avail Hrs P6 Hrs Avail Hrs P6 Hrs Avail Hrs P6 Hrs  ... P6 Hrs Avail Hrs   
0       40.0   12.0      14.0   10.0      35.0    6.0  ...    4.0      13.0   
1       37.0    2.0      21.0    2.0      15.0    0.0  ...    0.0       7.0   
2       26.0    2.0      18.0    2.0      20.0    2.0  ...    0.0      26.0   
3     -107.0    3.0     -62.0    3.0     -64.0    2.0  ...    1.0      -8.0   
4     -271.0    6.0    -239.0    5.0    -170.0    4.0  ...    3.0    -112.0   
5     -196.0   11.0    -108.0   11.0     -81.0   13.0  ...    4.0      86.0   
6      -43.0   10.0    -100.0    6.0      57.0    3.0  ...    1.0      75.0   
7     -308.0   22.0     -74.0   22.0       8.0   31.0  ...   21.0      24.0   
8     -318.0    2.0    -184.0    2.0    -153.0    2.0  ...    1.0     -99.0   
9        6.0    2.0       4.0    1.0       3.0    0.0  ...    0.0       9.0   
10      67.0    2.0      56.0    2.0      58.0    1.0  ...    0.0      70.0   
11       0.0    0.0       0.0    0.0       0.0    0.0  ...    0.0       0.0   
12       0.0   11.0       0.0   11.0       0.0   15.0  ...   10.0       0.0   
13      17.0    0.0      42.0    0.0     -28.0    0.0  ...    0.0     -85.0   
14      13.0    0.0       4.0    0.0     103.0    0.0  ...    0.0      94.0   
15      55.0    0.0      35.0    0.0      27.0    0.0  ...    0.0      38.0   
16      24.0    8.0     -22.0    5.0      98.0    1.0  ...    0.0     117.0   
17       0.0    5.0       0.0    4.0       0.0    0.0  ...    0.0       0.0   
18    -199.0    2.0    -153.0    2.0    -149.0    0.0  ...    0.0      22.0   
19     148.0    5.0     101.0    1.0     117.0    0.0  ...    0.0     140.0   
20     -16.0   11.0     -34.0    8.0      21.0    5.0  ...    4.0      42.0 

In [54]:
Columns_to_exclude = ['Resource_Name','Employee_code','Utilization_Target']
# data11.dtypes

In [55]:
# data11['Total_P6_Hours'] = data11.loc[:, ~data11.columns.isin(Columns_to_exclude)].sum(skipna=True)
subset = data11.loc[:, ~data11.columns.isin(Columns_to_exclude)].sum(axis=1)
data11['Total P6 Hours'] = subset
data11

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24,Total P6 Hours
0,"Cogger, Brittany A",114058,92,15.0,12.0,10.0,6.0,4.0,0.0,0.0,0.0,0.0,47.0
1,"Deboer, Shana L",23496,35,3.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
2,"Davis, Thomas W",24533,90,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,8.0
3,"Wood, Christine M",25974,60,4.0,3.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,13.0
4,"Taurasi, Paul M",40037,85,9.0,6.0,5.0,4.0,3.0,0.0,0.0,0.0,0.0,27.0
5,"Brick, William D",50480,87,14.0,11.0,11.0,13.0,4.0,0.0,0.0,0.0,0.0,53.0
6,"Jembere, Hana A",706527,96,13.0,10.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0,33.0
7,"Cailer, Noel",706796,70,26.0,22.0,22.0,31.0,21.0,0.0,0.0,0.0,0.0,122.0
8,"Dunbar, Caroline J",706873,63,3.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,10.0
9,"Kwong, Alison N",708617,10,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [56]:
data12 = data11[['Employee_code','Total P6 Hours']]
data12

,Employee_code,Total P6 Hours
0,114058,47.0
1,23496,7.0
2,24533,8.0
3,25974,13.0
4,40037,27.0
5,50480,53.0
6,706527,33.0
7,706796,122.0
8,706873,10.0
9,708617,6.0


In [57]:
av_hours2=pd.merge(av_hours1,data12, how='left')

In [53]:
av_hours2

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,...,Dec-24,Jan-25,Feb-25,Mar-25,Apr-25,May-25,Jun-25,Jul-25,Aug-25,Total P6 Hours
0,"Estrada, Jennifer A",102077,0,-95.200,-84.20,-30.90,-19.890,-16.15,-22.03,-27.340,...,-24.54,-17.810,-14.87,-5.73,-3.71,-3.71,-4.470,-3.49,-3.190,1.42
1,"Varghese, Joyson",114642,96,-472.020,-427.45,-422.84,-380.040,-203.21,-205.65,-249.090,...,114.41,93.920,99.34,148.25,130.02,133.49,160.150,128.13,126.830,215.61
2,"Woelke, Allen D",25744,45,-95.740,-58.09,-51.60,-41.910,-38.55,-0.80,19.000,...,50.22,37.450,36.76,43.87,37.73,36.48,45.740,39.92,43.280,36.46
3,"Pjetrovic, Indira S",33616,60,-536.410,-168.09,-152.06,-206.170,-146.48,-149.86,-196.730,...,-26.24,-81.660,-98.95,-163.76,-27.54,-89.46,14.060,17.24,15.180,52.50
4,"Plante, Stanley V (Stan)",35519,80,-53.910,-60.01,-52.35,27.630,59.97,73.20,74.000,...,152.90,123.930,123.91,154.88,123.91,124.38,155.850,124.71,124.790,0.00
5,"Opem, Paul J",50912,8,9.590,7.86,9.69,13.600,10.34,10.14,14.090,...,15.93,12.770,12.77,15.97,12.77,12.77,15.970,11.61,11.930,0.70
6,"Haire, Jerry W",701358,90,17.710,8.45,22.24,21.930,35.00,33.48,40.210,...,85.74,79.370,77.37,96.65,76.95,76.95,120.270,49.95,52.230,92.45
7,"Seiser, Madeline J",701794,95,72.050,14.48,-26.30,-45.430,6.95,52.15,64.590,...,75.97,86.330,91.69,94.60,97.98,103.50,125.800,100.03,101.340,142.41
8,"Rhames, Alan D",705435,80,-163.210,-88.43,-116.88,-124.860,-58.23,-15.51,20.150,...,6.18,15.380,11.40,22.42,46.32,45.58,93.760,94.09,85.870,30.00
9,"Kumari, Anamika",705746,95,-16.100,-90.72,-180.86,-217.410,-95.52,-28.59,0.530,...,115.03,99.170,105.35,158.49,128.31,130.54,165.580,138.49,138.060,124.82


In [54]:
# Compare av_hours & data10
com = data11.set_index(['Resource_Name','Employee_code','Utilization_Target','Total P6 Hours']).compare(av_hours2.set_index(['Resource_Name','Employee_code','Utilization_Target','Total P6 Hours']), align_axis = 1).rename(columns={'self': 'P6 Hrs', 'other': 'Avail Hrs'}, level=1).reset_index()

# Replace Nan with 0
co = com.replace(np.NaN, 0)
co=co.round(0)

In [55]:
co = co.sort_values('Total P6 Hours', ascending=False)

In [56]:
co

Resource_Name Employee_code Utilization_Target Total P6 Hours  \
                                                                               
26             Baig, Aasiyah        719271                 96          296.0   
28             Byrd, Brian G        719496                 96          270.0   
44                   Pari, N         91788                 85          226.0   
27            Khoury, Rana N        719301                 97          224.0   
1           Varghese, Joyson        114642                 96          216.0   
42          Voigt, Alexander         91575                 93          204.0   
19        Agniel, Samantha J        713582                 96          190.0   
17         Triana, Eugenio J        710906                 95          147.0   
7         Seiser, Madeline J        701794                 95          142.0   
25            Kajale, Atul N        718931                 93          139.0   
9            Kumari, Anamika        705746                 95          125.0   
22          Kamal, Shadman M        715035                 97          108.0   
29         Gilezan, Julian M        719875                 97          107.0   
15          Ahmed, Ishtiaque        710824                 97          101.0   
30              Soni, Chetan        719953                 70           99.0   
6             Haire, Jerry W        701358                 90           92.0   
23      Mostafazadeh, Mahnaz        717462                 93           86.0   
14     Al-Rahman, Shariyer M        710553                 97           84.0   
38            Khole, Sagar H         86686                 93           84.0   
12         Martinez, Wendy M        709486                 80           82.0   
31              Khan, Amal F        720485                 97           78.0   
43         V S, Rajesh Kumar         91591                 85           56.0   
3        Pjetrovic, Indira S         33616                 60           52.0   
33  MacLeod, Judith A (Judy)         83229                 55           51.0   
18           Hee, Konrad W H        712533                 75           40.0   
37        Burdett, Stephen M         86489                 65           38.0   
16             April, Gilles        710890                 86           36.0   
2            Woelke, Allen D         25744                 45           36.0   
10           Datta, Somenath        707876                 73           36.0   
21           Lynk, Charles N        714726                 97           32.0   
8             Rhames, Alan D        705435                 80           30.0   
41           Cormier, Shanna         88967                 60           30.0   
20          Pedarla, Aravind        713625                  0           20.0   
11           Ibrahim, Said Y        708762                 92           18.0   
40        Nicholson, David M         88350                 55           16.0   
39             Salva, Ivan G         88314                 92           13.0   
34          Frizzell, Carl S         84513                 77           13.0   
13           Hamad, Khidir A        710285                 65           12.0   
24        Bocanegra, Armando        718456                 97            9.0   
35         Stalnaker, Todd J         85420                 30            7.0   
36            Coyne, Wendy L         86319                 51            6.0   
32           Guemez, Sarah A         76045                 50            2.0   
5               Opem, Paul J         50912                  8            1.0   
0        Estrada, Jennifer A        102077                  0            1.0   
4   Plante, Stanley V (Stan)         35519                 80            0.0   

   Mar-24           Apr-24           May-24            ... Apr-25            \
   P6 Hrs Avail Hrs P6 Hrs Avail Hrs P6 Hrs Avail Hrs  ... P6 Hrs Avail Hrs   
26   11.0      85.0    9.0      72.0   23.0      93.0  ...    2.0     105.0   
28    0.0    -160

In [57]:
## color the missing employee to red:

In [58]:
missed_emp_list = miss_df['Employee_code'].to_list()
missed_emp_list

['713625']

In [59]:
co['Utilization_Target'] = co['Utilization_Target'].astype(object)

In [60]:
co['Utilization_Target'] = co['Utilization_Target'].astype(str) + '%'

In [61]:
co.rename(columns = {'Utilization_Target':'UT(%)'}, inplace = True)

In [ ]:
co.rename(columns = {'Resource_Name':'Resource Name'}, inplace = True)
co.rename(columns = {'Employee_code':'Employee No.'}, inplace = True)

In [62]:
def missed_emp(co, missed_emp):
    mask = co['Employee_code'].isin(missed_emp)
    co_style = co.style.apply(lambda x: ['color: red' if v else '' for v in mask], subset=['Resource_Name', 'Employee_code'])
    return co_style

def compare_and_color(row, compare, missed_emp_list):
    styles = [''] * len(row)
    for i in range(len(compare.columns)-1):
        col1 = compare.columns[i]
        col2 = compare.columns[i + 1]
        if col1[0] == col2[0]:
            if row[col2] - row[col1]> -40:
                styles[compare.columns.get_loc(col2)] = 'background-color: lightblue; color: black'
            elif -40 > row[col2] - row[col1] > -60:
                styles[compare.columns.get_loc(col2)] = 'background-color: yellow; color: black'
            elif -60 > row[col2] - row[col1] > -80:
                styles[compare.columns.get_loc(col2)] = 'background-color: orange; color: black'
            else:
                styles[compare.columns.get_loc(col2)] = 'background-color: red; color: black'
    return styles

# Apply missed_emp function
styled_co_missed_emp = missed_emp(co, missed_emp_list)

# Apply compare_and_color function
styled_co = styled_co_missed_emp.apply(compare_and_color, compare=co, missed_emp_list=missed_emp_list, axis=1).format(precision=1)

# Display styled DataFrame
styled_co

In [ ]:
## End

In [74]:
compare  = co.copy()

In [75]:
def compare_and_color(row, compare):
    styles = [''] * len(row)
    for i in range(len(compare.columns)-1):
        col1 = compare.columns[i]
        col2 = compare.columns[i + 1]
        if col1[0] == col2[0]:
            if row[col2] - row[col1]> -40:
                styles[compare.columns.get_loc(col2)] = 'background-color: lightblue; color: black'
            elif -40 > row[col2] - row[col1] > -60:
                styles[compare.columns.get_loc(col2)] = 'background-color: yellow; color: black'
            elif -60 > row[col2] - row[col1] > -80:
                styles[compare.columns.get_loc(col2)] = 'background-color: orange; color: black'
            else:
                styles[compare.columns.get_loc(col2)] = 'background-color: red; color: black'
    return styles

styled_df = co.style.apply(compare_and_color, compare=co, axis=1).format(precision=1)

In [58]:
# To get current directory
current_directory = os.getcwd()

In [59]:
global temp_folder
temp_folder=current_directory+'\\reports'
now = t.time()
for f in os.listdir(temp_folder):
    f = os.path.join(temp_folder, f)
    if os.stat(f).st_mtime < now - 1 * 86400:
         if os.path.isfile(f):
            os.remove(os.path.join(temp_folder, f))

In [125]:
# Coloring
def compare_and_color(row, compare):
    styles = [''] * len(row)
    for i in range(len(compare.columns)-1):
        col1 = compare.columns[i]
        col2 = compare.columns[i + 1]
        if col1[0] == col2[0]:
            if row[col2] - row[col1]> -40:
                styles[compare.columns.get_loc(col2)] = 'background-color: lightblue; color: black'
            elif -40 > row[col2] - row[col1] > -60:
                styles[compare.columns.get_loc(col2)] = 'background-color: yellow; color: black'
            elif -60 > row[col2] - row[col1] > -80:
                styles[compare.columns.get_loc(col2)] = 'background-color: orange; color: black'
            else:
                styles[compare.columns.get_loc(col2)] = 'background-color: red; color: black'
    return styles

styled_df = co.style.apply(compare_and_color, compare=co, axis=1).format(precision=1)
file_name = temp_folder+"/Resource Analysis Report-{}-{}.xlsx".format('nasreenn@cdmsmith.com',pn_list[0])
# Saving two dataframes 'final_data' & 'styled_df' in excel file with defined sheet name

with pd.ExcelWriter(file_name) as writer:
    styled_df.to_excel(writer, sheet_name = 'Summary')
#     styled_df.to_excel(writer, sheet_name = 'Overallocated Resources')

# Opening the excel file
workbook = openpyxl.load_workbook(file_name)
worksheet1 = workbook['Summary']
# worksheet2 = workbook["Overallocated Resources"]
list1 = data7[~data7['Employee_code'].isin(listrft)]
list2 = list(list1['Employee_code'].unique())
list3 = [i.strip() for i in list2]
worksheet1.cell(row = worksheet1.max_row + 1, column = 1).value = ""
x = worksheet1.cell(row = worksheet1.max_row + 1, column = 1)
x.value = '''ATTENTION - Resources not found in RFT'''
x.font = Font(bold = True, color = 'FF5733', italic = False)   # BLACK TEXT - 000000, RED - FF5733, Maroon - 800000
x.fill = PatternFill(start_color = "FFFFFF", fill_type = "solid")  # off white - FAF9F6, WHITE - FFFFFF
x = worksheet1.cell(row = worksheet1.max_row + 1, column = 1)
x.value = ''' {m} '''.format(m=list3)
x.font = Font(bold = True, color = '000000', italic = True)

# column width
column_widths = {'A': 22, 'B': 16, 'C': 16}
for column, width in column_widths.items():
    try:
        if column in ['A', 'B', 'C']:
            worksheet1.column_dimensions[column].width = width
    except:
        for column in worksheet1.columns:
            adjusted_width = 8
            worksheet1.column_dimensions[column[0].column_letter].width = adjusted_width

# Text alignment
max_row = worksheet1.max_row
max_column = worksheet1.max_column
for row in range(1, max_row + 1):
    for column in range(1, max_column + 1):
        if column != 1:
            cell = worksheet1.cell(row = row, column = column)
            alignment = Alignment(horizontal = 'center')
            cell.alignment = alignment
        else:
            pass

# Inserting image
img = Image('Summary-note.png')
img.width = 500
img.height = 150
col = "A"
row_num = str(len(comb['Employee_code']) + 7)
row_call = col + row_num
img.anchor = row_call
worksheet1.add_image(img)

# Coloring the headers and index
cols_color = PatternFill(patternType='solid', fgColor = Color('D8E4BC') )
rows_color = PatternFill(patternType='solid',fgColor=Color ('FFFFFF') )
for i in range(1,len(comb.columns) + 1):
    worksheet1.cell(row = 1, column = i).fill = cols_color
for i in range(2,len(comb.index)+2):
    worksheet1.cell(row = i, column = 1).fill = rows_color
    worksheet1.cell(row = i, column = 2).fill = rows_color
    worksheet1.cell(row = i, column = 3).fill = rows_color
worksheet1.column_dimensions['A'].hidden = True

workbook.save(file_name)
workbook.close()

In [62]:
# Coloring before
def compare_and_color(row, compare):
    styles = [''] * len(row)
    for i in range(len(compare.columns)-1):
        col1 = compare.columns[i]
        col2 = compare.columns[i + 1]
        if col1[0] == col2[0]:
            if row[col2] - row[col1]> -40:
                styles[compare.columns.get_loc(col2)] = 'background-color: lightblue; color: black'
            elif -40 > row[col2] - row[col1] > -60:
                styles[compare.columns.get_loc(col2)] = 'background-color: yellow; color: black'
            elif -60 > row[col2] - row[col1] > -80:
                styles[compare.columns.get_loc(col2)] = 'background-color: orange; color: black'
            else:
                styles[compare.columns.get_loc(col2)] = 'background-color: red; color: black'
    return styles

styled_df = co.style.apply(compare_and_color, compare=co, axis=1).format(precision=1)
file_name = temp_folder+"/Resource Analysis Report-{}-{}.xlsx".format('nasreenn@cdmsmith.com',pn_list[0])
# Saving two dataframes 'final_data' & 'styled_df' in excel file with defined sheet name
with pd.ExcelWriter(file_name) as writer:
#     final_data.to_excel(writer, sheet_name = 'Summary',index = False)
    styled_df.to_excel(writer, sheet_name = 'Overallocated Resources')

# Opening the excel file
workbook = openpyxl.load_workbook(file_name)
# worksheet1 = workbook['Summary']
worksheet2 = workbook["Overallocated Resources"]
list1 = data7[~data7['Employee_code'].isin(listrft)]
list2 = list(list1['Employee_code'].unique())
list3 = [i.strip() for i in list2]

# #### Overallocated Resources
# column width
column_widths = {'B': 20}
for column, width in column_widths.items():
    if column in ['B']:
        worksheet2.column_dimensions[column].width = width
    else: 
        for column in worksheet2.columns:
            adjusted_width = 25
            worksheet2.column_dimensions[column[0].column_letter].width = adjusted_width

# Text alignment
max_row = worksheet2.max_row
max_column = worksheet2.max_column
for row in range(1, max_row + 1):
    for column in range(1, max_column + 1):
        if column != 2:
            cell = worksheet2.cell(row = row, column = column)
            alignment = Alignment(horizontal = 'center')
            cell.alignment = alignment
        else:
            pass
new_serial_numbers = list(range(1,len(compare['Resource_Name'])+1))  # Replace this with your desired new serial numbers

# Iterate through the rows and update the serial numbers in column A
for i, new_value in enumerate(new_serial_numbers):
    row_number = i + 4  # Rows are 1-indexed in Excel
    worksheet2.cell(row = row_number, column=1, value=new_value)
worksheet2.column_dimensions['A'].hidden = True
worksheet2.delete_rows(idx=3)   #for rows

# Inserting image
img = Image('color_legend.png')
img.width = 500
img.height = 35
col1 = "A"
row_num1 = str(len(compare['Resource_Name']) + 5)
row_call1 = col1 + row_num1
img.anchor = row_call1
worksheet2.add_image(img)

# Inserting image -- 
img = Image('OA-note.png')
img.width = 500
img.height = 100
col2 = "A"
row_num2 = str(len(compare['Resource_Name']) + 8)
row_call2 = col2 + row_num2
img.anchor = row_call2
worksheet2.add_image(img)

# Coloring the headers and index
col_color = PatternFill(patternType='solid', fgColor = Color('D8E4BC') )
#row_color = PatternFill(patternType='solid',fgColor=Color ('E0EEEE') )
for i in range(1,len(compare.columns) + 2):
    worksheet2.cell(row = 1, column = i).fill = col_color
workbook.save(file_name)
workbook.close()

In [ ]:
#------------------------------CODE ENDS HERE--------------------------------

In [4]:
# Popping the column to change the position
cll1 = rft_report.pop('Utilization_Target')
cll2 = rft_report.pop('Employee_code')
cll3 = rft_report.pop('Resource_Name')
cll4 = rft_report.pop('Project Number')

# Placing the pop up column at first
rft_report.insert(2, 'Utilization_Target', cll1)
# Placing the pop up column at first
rft_report.insert(2, 'Employee_code', cll2)
# Placing the pop up column at first
rft_report.insert(2, 'Resource_Name', cll3)
# Placing the pop up column at first
rft_report.insert(2, 'Project Number', cll4)

# To replace dash (-) & NaN with Zero (0)
except_columns = ['Project', 'Resource (Utilization Target %)/Job Code', 'Project Number', 'Resource_Name', 'Employee_code', 'Utilization_Target']
for column in rft_report.columns:
    if column not in except_columns:
        rft_report[column] = rft_report[column].replace('-', 0)
        rft_report[column] = rft_report[column].replace(np.NaN, 0)

# rft_report.isnull().sum()
# To change the dtypes of columns
except_columns = ['Project', 'Resource (Utilization Target %)/Job Code', 'Project Number', 'Resource_Name', 'Employee_code', 'Utilization_Target']
for column in rft_report.columns:
    if column not in except_columns:
        rft_report[column] = pd.to_numeric(rft_report[column], errors = 'coerce')

In [5]:
# rft_report

In [6]:
# Dropping unwanted columns
data1 = data.drop(['Activity Name', 'Curve', 'Spreadsheet Field', 'Remaining Units'], axis = 1)

#changing the format of date and adding it in a new column
calendar['Month_Year'] = calendar['GL Period'].apply(lambda x: x.strftime('%b-%y')) 
# Extracting Month columns 
data2 = data1.iloc[:, 1:]

x = data2.columns[0]
x2 = x.strftime('%Y-%m-%d %H:%M:%S')
x3 = x2.replace(' 00:00:00', '')
# Rename a column using the .rename() method
data2.rename(columns={x2: x3}, inplace=True)

In [7]:
# data2

In [8]:
# Fix time frame according to RFT (18 months)

# To get currentatime
current_date = dt.datetime.now()
# current_date = '2023-12-31'
# Select dates less than current time
current_date_less = calendar[calendar['W/E Date'] <= current_date]
# Convert the datatype
current_date_less['W/E Date'] = current_date_less['W/E Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
# Select last date which is just less than current date - start date
start_d = current_date_less.tail(1)

# Store start date in a list
start = start_d['GL Period'].tolist()
start_dt = start[0]
varr = start_dt.strftime('%Y-%m-%d')
start_date = varr.replace(' 00:00:00', '')

In [9]:
# start_date

In [10]:
# Convert the string to a datetime object
start_date = dt.datetime.strptime(start_date, '%Y-%m-%d').date()

# Add 18 months using relativedelta
end_date = start_date + relativedelta(months=17)

# Convert the future date back to a string if needed
end_date = end_date.strftime('%Y-%m-%d')

var1 = data2.columns[0]
var2 = var1.strftime('%Y-%m-%d')

# Specify the format of the date string
date_format = '%Y-%m-%d'  # This format corresponds to 'YYYY-MM-DD'

In [11]:
# end_date

In [12]:
# Convert the string to a datetime.date object
date_obj = datetime.strptime(var2, date_format).date()
end_date = datetime.strptime(end_date, date_format).date()

# Create a time object with the desired time (e.g., midnight)
time_obj = time(0, 0)  # This represents 00:00:00 (midnight)

# Combine the date and time to create a datetime.datetime object
datetime_obj = datetime.combine(date_obj, time_obj)
datetime_start = datetime.combine(start_date, time_obj)
datetime_end = datetime.combine(end_date, time_obj)

In [13]:
# # Format the datetime object as 'Dec-YY'
# month_year = datetime_start.strftime('%b-%y')
# month_year
# datetime_start

In [14]:
# # Selecting columns mentioned in P6
# calendar = calendar[calendar['W/E Date'].isin(data2.columns)] 
# calender = calendar[calendar['GL Period'] >= datetime_start]
# calender

In [17]:
# calender = calender.reset_index(drop = True)
# # Select rows within the date range
# final_calender = calender[(calender['GL Period'] >= datetime_start) & (calender['GL Period'] <= datetime_end)]
# column_list = final_calender['W/E Date'].tolist()
# data2 = data2[column_list]
# #extracting all month-year values
# month = final_calender['Month_Year'].values
# data5 = data2.set_axis(month, axis = 'columns')

In [18]:
# calendar

In [19]:
if datetime_start != datetime_obj:
        if datetime_start < datetime_obj:
            # Selecting columns mentioned in P6
            calendar = calendar[calendar['W/E Date'].isin(data2.columns)] 
            calender = calendar[calendar['W/E Date'] >= datetime_obj]
            # Reindexing
            calender = calender.reset_index(drop = True)
            # Select rows within the date range
            final_calender = calender[(calender['W/E Date'] >= datetime_obj) & (calender['W/E Date'] <= datetime_end)]
            column_list = final_calender['W/E Date'].tolist()
            # Convert the list of timestamps to a list of strings
            col_list = [timestamp.strftime('%Y-%m-%d') for timestamp in column_list]
            data2 = data2[col_list]
            #extracting all month-year values
            month = final_calender['Month_Year'].values
            data5 = data2.set_axis(month, axis = 'columns')
        else: 
            # Selecting columns mentioned in P6
            calendar = calendar[calendar['W/E Date'].isin(data2.columns)] 
            calender = calendar[calendar['GL Period'] >= datetime_start]
            # Reindexing
            calender = calender.reset_index(drop = True)
            # Select rows within the date range
            final_calender = calender[(calender['GL Period'] >= datetime_start) & (calender['GL Period'] <= datetime_end)]
            column_list = final_calender['W/E Date'].tolist()
            data2 = data2[column_list]
            #extracting all month-year values
            month = final_calender['Month_Year'].values
            data5 = data2.set_axis(month, axis = 'columns')
else: 
    # Selecting columns mentioned in P6
    calendar = calendar[calendar['W/E Date'].isin(data2.columns)] 
    calender = calendar[calendar['W/E Date'] >= datetime_obj]
    # Reindexing
    calender = calender.reset_index(drop = True)
    # Select rows within the date range
    final_calender = calender[(calender['W/E Date'] >= datetime_obj) & (calender['W/E Date'] <= datetime_end)]
    column_list = final_calènder['W/E Date'].tolist()
    data2 = data2[column_list]
    #extracting all month-year values
    month = final_calender['Month_Year'].values
    data5 = data2.set_axis(month, axis = 'columns')

In [20]:
# data5

In [21]:
# Joining df with p_six
data5.insert(loc = 0, column = 'Activity ID', value = data['Activity ID'])

# Extracting rows with 'Resource Name'
names = data5[data5['Activity ID'].str.contains("Resource")]

# Creating new column named 'Resource Name' by replacing Resource name from col 'Activity ID'
names['Resource Name'] = names['Activity ID'].str.replace('Resource Name: ', '')

# Dropping unwanted columns
dataset = names.drop('Activity ID', axis = 1)

# Popping the column to change the position
val1 = dataset.pop('Resource Name')

# Placing the pop up column at first
dataset.insert(0, 'Resource Name', val1)

# Reseting Index
data6 = dataset.reset_index()

# Dropping unwanted columns
data7 = data6.drop(['index'], axis = 1)

# Changing column name '0' ---> 'Project Number'
data7.rename(columns = {'Resource Name': 'Resource_Name'}, inplace = True)

# Remove extra space
data7['Resource_Name'] = data7['Resource_Name'].apply(str.strip)

# Extracting rows with 'WBS'
wbs = data5[data5['Activity ID'].str.contains("WBS: ")]

# Extracting numbers from main column
wbs['num'] = wbs['Activity ID'].str.findall(r'[0-9]+')   #output: [285241, 01, 01], want only oth index i.e., project number

# Removing brackets from each row
wbs["Project Number"] = wbs["num"].str.get(0)

# Project Number in p6
global pn_list
pn_list = list(wbs['Project Number'].unique())
    # List of unique Resource Name in p6 data
unique_resource = list(data7['Resource_Name'].unique())
unique_resource = [i.strip() for i in unique_resource]

In [22]:
# data7

In [23]:
# rft_report[rft_report['Resource_Name'] == 'Karthik, Jayanthi M']
# merged_df
# unique_resource

In [24]:
# Select p6 resource from rft for 'Employee code and UT'
rft1 = rft_report[rft_report['Resource_Name'].isin(unique_resource)]

# Get unique list of code and ut for resource in p6
rft2 = rft1[['Resource_Name', 'Employee_code', 'Utilization_Target']].drop_duplicates()
rft2 = rft2.reset_index()

# Dropping unwanted columns
rft3 = rft2.drop(['index'], axis = 1)

# Merging data7 & rft3 for 'Employee_code' & 'Utilization_Target'
merged_df = pd.merge(data7, rft3, on = 'Resource_Name', how = 'right')

# Popping the column to change the position
vll1 = merged_df.pop('Utilization_Target')
vll2 = merged_df.pop('Employee_code')

# Placing the pop up column at first
merged_df.insert(1, 'Utilization_Target', vll1)
# Placing the pop up column at first
merged_df.insert(1, 'Employee_code', vll2)

# To replace dash (-) & NaN with Zero (0)
except_col = ['Resource_Name', 'Employee_code', 'Utilization_Target Name']
for column in merged_df.columns:
    if column not in except_col:
        #data9[column] = data9[column].replace('-', 0)
        merged_df[column] = merged_df[column].replace(np.NaN, 0)

# Popping the column to change the position
cal1 = ut.pop('Utilization_Target')
cal2 = ut.pop('Employee_code')
cal3 = ut.pop('Resource_Name')

In [25]:
# rft1[rft1['Resource_Name'] == 'Karthik, Jayanthi M']
# merged_df

In [26]:
# Placing the pop up column at first
ut.insert(1, 'Utilization_Target', cal1)
# Placing the pop up column at first
ut.insert(1, 'Employee_code', cal2)
# Placing the pop up column at first
ut.insert(1, 'Resource_Name', cal3)
ut = ut.drop('Utilization_Target_percent', axis = 1)

## Select specific uts
sel = ut[ut['Resource_Name'].isin(unique_resource)]

# Ut with only combine columns
ut_com = sel[merged_df.columns]

# RESET INDEX
ut_com = ut_com.reset_index(drop = True)

In [27]:
# ut_com

In [28]:
# Groupby RFT_REPORT based on 'Resource Name' & 'Project Number'
grouped = rft_report.groupby(['Resource_Name', 'Employee_code', 'Utilization_Target','Project Number']).sum()

# Redexing to apply filter on 'Resource Name'
grouped_rft = grouped.reset_index()

# P6 in RFT
set1 = grouped_rft[grouped_rft['Resource_Name'].isin(unique_resource)]

In [29]:
# set1

In [30]:
# Selecting rft based on Project number not in final_p6 (NOT EXIST/EXIST CASE)
set2 = set1[~set1['Project Number'].isin(pn_list)]

# set2 with only merged_df columns ---index issue
set2 = set2[merged_df.columns]

# Groupby RFT_REPORT based on 'Resource Name' & 'Project Number'
set3 = set2.groupby(['Resource_Name', 'Employee_code', 'Utilization_Target']).sum()
set3 = set3.reset_index()

## 1. Conbine data9 and set3
combine = merged_df.set_index(['Resource_Name','Employee_code','Utilization_Target']).add(set3.set_index(['Resource_Name','Employee_code','Utilization_Target']), fill_value = 0.0).reset_index()

In [79]:
set3

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,...,Nov-24,Dec-24,Jan-25,Feb-25,Mar-25,Apr-25,May-25,Jun-25,Jul-25,Aug-25
0,"Agniel, Samantha J",713582,96,108.240,119.53,127.69,247.760,46.13,49.18,157.640,...,46.82,153.63,47.190,3.24,1.63,4.32,4.32,4.030,0.45,0.470
1,"Ahmed, Ishtiaque",710824,97,124.500,95.24,85.61,120.690,73.76,55.56,68.950,...,34.92,38.65,45.490,49.31,61.66,49.31,49.31,51.670,35.92,23.760
2,"Al-Rahman, Shariyer M",710553,97,202.810,162.06,121.96,160.710,100.80,77.45,88.170,...,41.53,40.05,14.750,11.19,13.95,11.19,11.19,14.380,11.54,12.160
3,"April, Gilles",710890,86,133.750,97.01,97.01,89.200,32.41,34.83,41.800,...,34.83,40.10,41.560,43.75,54.69,38.23,35.83,23.150,19.21,65.390
4,"Baig, Aasiyah",719271,96,107.290,81.88,60.12,61.120,27.09,17.73,21.740,...,2.00,1.98,41.250,48.32,60.39,48.32,48.32,57.970,44.69,1.800
5,"Bocanegra, Armando",718456,97,108.410,67.53,67.53,81.050,2.95,5.10,7.160,...,3.10,3.42,22.950,0.78,0.00,0.00,0.00,0.000,0.00,0.000
6,"Burdett, Stephen M",86489,65,177.750,108.44,96.90,133.810,95.73,108.51,98.230,...,65.33,75.65,68.360,67.94,72.89,59.66,56.71,69.960,54.91,56.320
7,"Byrd, Brian G",719496,96,351.840,263.63,234.89,260.500,344.92,404.64,289.220,...,326.18,213.28,95.250,103.87,92.67,138.23,139.60,159.430,122.37,83.360
8,"Cormier, Shanna",88967,60,270.020,193.92,193.18,229.520,170.33,157.83,163.090,...,103.58,116.79,91.560,86.04,99.96,71.15,70.20,85.590,59.27,44.070
9,"Coyne, Wendy L",86319,51,258.705,218.88,221.10,250.415,133.68,105.07,81.115,...,61.49,59.04,31.535,38.95,69.58,31.60,20.10,26.795,20.97,16.755


In [ ]:
# combine

In [33]:
# List of unique Resource Name in p6 data
listrft = list(combine['Resource_Name'].unique())
listrft = [i.strip() for i in listrft]
combine1 = ut_com.set_index(['Resource_Name','Employee_code','Utilization_Target']).subtract(combine.set_index(['Resource_Name','Employee_code','Utilization_Target']), fill_value=0).reset_index()

# List of missing resource name (not in rft)
resource = list(combine1['Resource_Name'].unique())
resource = [i.strip() for i in resource]

# Not in rft
missing_resource = set(unique_resource) - set(resource)
miss_df = data7[data7['Resource_Name'].isin(missing_resource)]

In [34]:
# miss_df

In [35]:
# Merging data7 & rft3 for 'Employee_code' & 'Utilization_Target'
comb = pd.concat([combine1, miss_df], axis = 0)
comb['Utilization_Target'] = comb['Utilization_Target'].astype(str)
comb['Employee_code'] = comb['Employee_code'].astype(str)

In [36]:
# comb

In [37]:
# To replace dash (-) & NaN with Zero (0)
except_col = ['Employee_code', 'Utilization_Target']
for column in comb.columns:
    if column in except_col:
        #data9[column] = data9[column].replace('-', 0)
        comb[column] = comb[column].replace('nan', '-')
comb=comb.round(0)
comb = comb.reset_index()
comb = comb.drop('index', axis=1)

def highlight_max(cell_value):
    highlight = 'background-color: tomato;'
    color = 'background-color: lightblue;'
    default = ''
    if type(cell_value) in [float]:
        if cell_value < 0:
            return highlight
        else:
            return color
    return default

# Apply color to combine
final_data = comb.style.applymap(highlight_max).format(precision = 0)
test = comb.copy()

# AVAILABLE RESOURCES
test = test[(test[test.columns[3:]] >= 0).all(axis=1)]
pos_resources = test['Resource_Name'].unique()

In [38]:
# final_data

In [78]:
# Selecting pos_resources from arr
neg_info = combine1[~combine1['Resource_Name'].isin(pos_resources)]
neg_info = neg_info.reset_index(drop=True)
neg_info

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,...,Nov-24,Dec-24,Jan-25,Feb-25,Mar-25,Apr-25,May-25,Jun-25,Jul-25,Aug-25
0,"Agniel, Samantha J",713582,96,-114.300,-124.38,-137.25,-263.000,-62.19,-60.76,-170.680,...,-59.52,-168.48,-64.070,-35.35,-3.85,-6.97,-7.11,-7.380,-3.10,-1.030
1,"Agniel, Samantha J",713582,97,194.000,155.20,155.20,194.000,155.20,155.20,194.000,...,155.20,194.00,155.200,155.20,194.00,155.20,155.20,194.000,155.20,155.200
2,"Ahmed, Ishtiaque",710824,97,68.360,59.05,63.70,62.260,64.69,90.21,115.100,...,116.79,150.53,102.430,94.67,132.34,104.12,104.03,140.100,117.51,131.070
3,"Al-Rahman, Shariyer M",710553,97,-26.560,-21.06,19.04,16.250,40.91,70.65,105.830,...,113.67,153.95,140.450,144.01,180.05,144.01,144.01,179.620,143.66,143.040
4,"April, Gilles",710890,85,170.000,136.00,136.00,170.000,136.00,136.00,170.000,...,136.00,170.00,136.000,136.00,170.00,136.00,136.00,170.000,136.00,136.000
5,"April, Gilles",710890,86,-140.000,-97.74,-98.28,-91.200,-36.85,-43.39,-44.800,...,-34.83,-42.60,-44.060,-48.75,-54.69,-38.23,-35.83,-23.150,-19.21,-65.390
6,"Baig, Aasiyah",719271,96,73.730,62.57,70.28,91.780,88.35,105.67,138.900,...,137.31,173.16,95.410,69.50,131.61,103.51,103.42,131.800,107.14,151.430
7,"Bocanegra, Armando",718456,94,188.000,150.40,150.40,188.000,150.40,150.40,188.000,...,150.40,188.00,150.400,150.40,188.00,150.40,150.40,188.000,150.40,150.400
8,"Bocanegra, Armando",718456,97,-109.500,-68.40,-68.40,-82.090,-3.78,-5.53,-7.160,...,-4.33,-4.77,-23.750,-0.78,0.00,0.00,0.00,0.000,0.00,0.000
9,"Burdett, Stephen M",86489,65,-60.160,-12.12,3.17,-8.530,4.22,-6.86,31.770,...,38.67,54.35,35.640,36.06,56.82,43.82,46.77,59.410,48.60,47.160


In [40]:
# neg_info

In [41]:
# Calling from rft: set3
rft_set4 = set3[set3['Resource_Name'].isin(neg_info['Resource_Name'].unique())].reset_index(drop=True)

In [42]:
# rft_set4

In [43]:
# Calling from p6: data9
data10 = merged_df[merged_df['Resource_Name'].isin(neg_info['Resource_Name'].unique())].reset_index(drop=True)
data10 = data10.sort_values(by = 'Resource_Name')

In [77]:
data10

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,...,Nov-24,Dec-24,Jan-25,Feb-25,Mar-25,Apr-25,May-25,Jun-25,Jul-25,Aug-25
18,"Estrada, Jennifer A",102077,0,0.30,0.24,0.24,0.29,0.23,0.12,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
15,"Woelke, Allen D",25744,45,1.30,2.40,1.31,1.52,0.41,1.09,1.13,...,2.46,3.35,2.96,2.00,0.80,3.00,3.09,3.70,2.93,1.78
0,"Pjetrovic, Indira S",33616,60,12.50,1.45,2.55,4.00,4.44,9.56,4.00,...,0.00,3.00,3.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00
7,"Plante, Stanley V (Stan)",35519,80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
20,"Rhames, Alan D",705435,80,0.00,1.45,2.55,4.00,0.00,4.00,4.00,...,0.00,3.00,3.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00
21,"Kumari, Anamika",705746,95,28.82,18.39,11.89,14.27,11.20,5.83,0.00,...,4.92,5.42,3.20,0.00,1.78,3.23,3.23,3.88,3.07,3.23
11,"Datta, Somenath",707876,73,2.42,1.94,1.99,2.44,1.93,1.55,1.29,...,2.22,2.44,2.19,2.50,2.05,1.87,1.89,2.27,1.80,1.52
5,"Ibrahim, Said Y",708762,92,0.00,0.00,0.76,1.52,1.21,1.26,1.38,...,0.63,0.89,2.16,5.56,0.00,0.44,0.47,0.56,0.44,0.09
22,"Hamad, Khidir A",710285,65,0.00,0.00,0.76,1.52,1.21,0.51,0.25,...,1.27,1.21,0.44,3.56,0.00,0.00,0.00,0.00,0.00,0.00
2,"Al-Rahman, Shariyer M",710553,97,17.75,14.20,14.20,17.04,13.49,7.10,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [45]:
# Calling from ut: ut_com
ut1 = ut_com[ut_com['Resource_Name'].isin(neg_info['Resource_Name'].unique())].reset_index(drop=True)

In [46]:
# ut1

In [47]:
## 1. Combine data10 and rft_set4
av_hours = ut1.set_index(['Resource_Name','Employee_code','Utilization_Target']).subtract(rft_set4.set_index(['Resource_Name','Employee_code','Utilization_Target']), fill_value=0).reset_index()
av_hours = av_hours.sort_values(by = 'Resource_Name')

In [80]:
av_hours

,Resource_Name,Employee_code,Utilization_Target,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,...,Nov-24,Dec-24,Jan-25,Feb-25,Mar-25,Apr-25,May-25,Jun-25,Jul-25,Aug-25
6,"Estrada, Jennifer A",102077,0,-95.200,-84.20,-30.90,-19.890,-16.15,-22.03,-27.340,...,-22.33,-24.54,-17.810,-14.87,-5.73,-3.71,-3.71,-4.470,-3.49,-3.190
24,"Woelke, Allen D",25744,45,-95.740,-58.09,-51.60,-41.910,-38.55,-0.80,19.000,...,33.45,50.22,37.450,36.76,43.87,37.73,36.48,45.740,39.92,43.280
18,"Pjetrovic, Indira S",33616,60,-536.410,-168.09,-152.06,-206.170,-146.48,-149.86,-196.730,...,-69.90,-26.24,-81.660,-98.95,-163.76,-27.54,-89.46,14.060,17.24,15.180
19,"Plante, Stanley V (Stan)",35519,80,-53.910,-60.01,-52.35,27.630,59.97,73.20,74.000,...,110.07,152.90,123.930,123.91,154.88,123.91,124.38,155.850,124.71,124.790
20,"Rhames, Alan D",705435,80,-163.210,-88.43,-116.88,-124.860,-58.23,-15.51,20.150,...,4.98,6.18,15.380,11.40,22.42,46.32,45.58,93.760,94.09,85.870
14,"Kumari, Anamika",705746,95,-16.100,-90.72,-180.86,-217.410,-95.52,-28.59,0.530,...,87.31,115.03,99.170,105.35,158.49,128.31,130.54,165.580,138.49,138.060
5,"Datta, Somenath",707876,73,-128.320,-83.78,-66.15,-59.740,-36.75,-32.17,-7.480,...,19.90,44.76,47.320,54.44,77.25,66.55,73.63,104.100,90.21,89.270
11,"Ibrahim, Said Y",708762,92,16.560,-19.33,12.39,92.780,116.62,95.58,136.550,...,122.61,167.20,133.790,132.10,151.15,120.78,128.16,164.060,135.92,127.700
9,"Hamad, Khidir A",710285,65,-119.420,-17.49,-19.65,-5.090,-24.46,11.12,29.250,...,26.19,65.12,50.620,49.14,69.52,47.36,48.26,59.800,78.13,70.710
0,"Al-Rahman, Shariyer M",710553,97,-8.810,-6.86,33.24,33.290,54.40,77.75,105.830,...,113.67,153.95,140.450,144.01,180.05,144.01,144.01,179.620,143.66,143.040


In [49]:
# Compare av_hours & data10
compare = data10.set_index(['Resource_Name']).compare(av_hours.set_index(['Resource_Name']), align_axis = 1).rename(columns={'self': 'P6 Hrs', 'other': 'Avail Hrs'}, level=1).reset_index()

# Replace Nan with 0
compare = compare.replace(np.NaN, 0)
compare=compare.round(0)

In [50]:
compare

In [70]:
pn_list[0]

'292279'

In [103]:
# Coloring
def compare_and_color(row):
    styles = [''] * len(row)
    for i in range(len(compare.columns)-1):
        col1 = compare.columns[i]
        col2 = compare.columns[i + 1]
        if col1[0] == col2[0]:
            if row[col1] > row[col2]:
                styles[compare.columns.get_loc(col1)] = 'background-color: pink; color: blue'
                styles[compare.columns.get_loc(col2)] = 'background-color: tomato; color: black'
            else:
                styles[compare.columns.get_loc(col1)] = 'background-color: lightblue; color: black'
                styles[compare.columns.get_loc(col2)] = 'background-color: lightblue; color: black'
    return styles
styled_df = compare.style.apply(compare_and_color, axis = 1).format(precision = 1)
file_name=temp_folder+"/Resource Analysis Report-{}-{}.xlsx".format(email,pn_list[0])
# Saving two dataframes 'final_data' & 'styled_df' in excel file with defined sheet name
with pd.ExcelWriter(file_name) as writer:
    final_data.to_excel(writer, sheet_name = 'Summary', index = False)
    styled_df.to_excel(writer, sheet_name = 'Overallocated Resources')

NameError: name 'temp_folder' is not defined